## Testing notebook for beta reconstruction software

For testing use the latest version of defdap on the grain_boundaies branch

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

from defdap.quat import Quat

from defdap.plotting import MapPlot

from beta_reconstruction.reconstruction import do_reconstruction, load_map

%matplotlib notebook

## Load in EBSD file
The `load_map` function will load the EBSD map from the specified file and
do prerequisite calculations such as finding grains and grain boundaries and
constructing a neighbour network of the grain boundaries.

In [ ]:
ebsd_file_path = "example_data/ZrNb_triplepoint.ctf"

EbsdMap = load_map(ebsd_file_path)

## Calculate possible beta orientations 

Possible beta orientations are calculated from the misorientations between each grain and its neighbours

In [ ]:
do_reconstruction(EbsdMap, burg_tol=5., ori_tol=3.)

The results are stored in the grain objects which comprise the EBSD map.

In [ ]:
EbsdMap.locateGrainID()

In [ ]:
grain = EbsdMap[8]

In [ ]:
grain.betaOris

In [ ]:
grain.possibleBetaOris

In [ ]:
np.array(grain.betaDeviations) *180 /np.pi

In [ ]:
grain.variantCount

burgers tol for each variant count - this will have to wait until we rewrtite the variant counting code

In [ ]:
possibleBetaOris = [item for sublist in grain.possibleBetaOris for item in sublist]

directions = [
    np.array([1,0,0]), 
    np.array([0,1,0]), 
    np.array([0,0,1])
]
markerSize = 100

fig, axes = plt.subplots(1, len(directions))

for direction, ax in zip(directions, axes):
    plot = Quat.plotIPF(grain.betaOris, direction, "cubic", marker='o', s=markerSize, fig=fig, ax=ax)
    Quat.plotIPF(possibleBetaOris, direction, "cubic", s=markerSize, plot=plot)

fig.tight_layout()

## Find the most common variant for each grain and set this as the beta orientaion

In [ ]:
modeVariants = []
parentBetaOris = []

for grain in EbsdMap:
#     modeVariant = np.argmax(grain.variantCount)
#     parentBetaOri = grain.betaOris[modeVariant]
    
    variantCount = np.array(grain.variantCount)
    # argmax would only pass one value if multiple variants have the same number of counts
    modeVariant = np.where(variantCount == np.max(variantCount))[0]
    if len(modeVariant) == 1:
        modeVariant = modeVariant[0]
        parentBetaOri = grain.betaOris[modeVariant]
    else:
        # multiple variants with same max
        modeVariant = -1
        parentBetaOri = Quat(1., 0., 0., 0.)
    
    modeVariants.append(modeVariant)
    grain.modeVariant = modeVariant
    parentBetaOris.append(parentBetaOri)
    grain.parentBetaOri = parentBetaOri

## Plot

### Vaiant number 

In [ ]:
plot = EbsdMap.plotGrainDataMap(grainData=modeVariants, vmin=-1, vmax=5, cmap="Set1")
plot.addColourBar("mode variant")

### IPF of grains filled with average orientation

In [ ]:
directions = [
    np.array([1,0,0]),
    np.array([0,1,0]),
    np.array([0,0,1])
]

fig, axes = plt.subplots(1, len(directions), figsize=(15,5))

for ax, direction in zip(axes, directions):
    betaGrainIPFColours = Quat.calcIPFcolours(parentBetaOris, direction, "cubic")

    EbsdMap.plotGrainDataMap(grainData=betaGrainIPFColours, fig=fig, ax=ax)
    
fig.tight_layout()

### Calculate and then plot beta orientation map

In [ ]:
from beta_reconstruction.crystal_relations import unq_hex_syms, burg_trans

transformations = []
for sym in unq_hex_syms:
    transformations.append(burg_trans * sym.conjugate)
    
variantMap = EbsdMap.grainDataToMapData(modeVariants, bg=-2)
    
betaQuatArray = np.empty_like(EbsdMap.quatArray)
for i in range(EbsdMap.yDim):
    for j in range(EbsdMap.xDim):
        variant = variantMap[i, j]
        if variant < 0:
            # points not part of a grain (-2) and 
            # those that were not reconstructed (-1)
            betaQuatArray[i, j] = Quat(1, 0, 0, 0)
        else:
#             betaQuatArray[i, j] = burg_trans * unq_hex_syms[variant].conjugate * EbsdMap.quatArray[i, j]
            betaQuatArray[i, j] = transformations[variant] * EbsdMap.quatArray[i, j]

In [ ]:
directions = [
    np.array([1,0,0]),
    np.array([0,1,0]),
    np.array([0,0,1])
]

fig, axes = plt.subplots(1, len(directions), figsize=(15,5))

for ax, direction in zip(axes, directions):
    betaIPFColours = Quat.calcIPFcolours(betaQuatArray.flatten(), direction, "cubic")
    # reshape back to map shape array
    betaIPFColours = np.reshape(betaIPFColours, (EbsdMap.yDim, EbsdMap.xDim, 3))
    
    # recolour the -1 and -2 variants
    # -1 grains not reconstructed (white)
    # -2 clusters too small to be a grain (black)
    indxs = np.where(variantMap == -1)
    betaIPFColours[indxs[0], indxs[1]] = np.array([1, 1, 1])
    indxs = np.where(variantMap == -2)
    betaIPFColours[indxs[0], indxs[1]] = np.array([0, 0, 0])
    
    plot = MapPlot.create(EbsdMap, betaIPFColours, fig=fig, ax=ax)
    
fig.tight_layout()